In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *

## Diagonal Part

In [2]:
totHam = pn**2 + qn**2

In [3]:
totHam

p_{n}**2 + q_{n}**2

In [4]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}

# warning be careful of array indexing...   
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1]
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {qs[0]: 0, qs[N+1]: 0}

In [5]:
s=''
for q in qs:
    s+='{}  '.format(q)
print(s)
qs[0]

q_{-1}  q_{0}  q_{1}  q_{2}  q_{3}  


q_{-1}

In [6]:
ham=0
for i in range(1,N+1):
    ham+=totHam.subs({qn: qs[i], pn: ps[i]}).subs(boundaryConditions)

ham

p_{0}**2 + p_{1}**2 + p_{2}**2 + q_{0}**2 + q_{1}**2 + q_{2}**2

In [7]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

a^{\dagger}_{0}*a_{0} + a^{\dagger}_{1}*a_{1} + a^{\dagger}_{2}*a_{2} + a_{0}*a^{\dagger}_{0} + a_{1}*a^{\dagger}_{1} + a_{2}*a^{\dagger}_{2}

In [8]:
print(hoHam.args[0])
for a in hoHam.args[0].args:
    print(a.name)

a^{\dagger}_{0}*a_{0}
a^{\dagger}_{0}
a_{0}


In [9]:
convert_boson_term_to_matrix(hoHam.args[0],4,3,aops,adags)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 3.00000000000000, 0, 0],
       [0, 0, 0, ..., 0, 3.00000000000000, 0],
       [0, 0, 0, ..., 0, 0, 3.00000000000000]], dtype=object)

In [14]:
cutoff=4
hamMat=convert_boson_to_matrix(hoHam,cutoff,N,aops,adags)

In [15]:
hamMat

array([[3.00000000000000, 0, 0, ..., 0, 0, 0],
       [0, 5.00000000000000, 0, ..., 0, 0, 0],
       [0, 0, 7.00000000000000, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 9.00000000000000, 0, 0],
       [0, 0, 0, ..., 0, 11.0000000000000, 0],
       [0, 0, 0, ..., 0, 0, 9.00000000000000]], dtype=object)

In [16]:
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

array([ 3.+0.j,  5.+0.j,  5.+0.j,  5.+0.j,  5.+0.j,  5.+0.j,  5.+0.j,
        7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,
        7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,  7.+0.j,
        7.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,
        9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,
        9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,  9.+0.j,
       11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j,
       11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j, 11.+0.j,
       11.+0.j, 13.+0.j, 13.+0.j, 13.+0.j, 13.+0.j, 13.+0.j, 13.+0.j,
       15.+0.j], dtype=complex64)

In [13]:
cutoff=8
term=hoHam.args[0]+hoHam.args[3]
print(term)
hamMat=convert_boson_to_matrix(term,cutoff,1,aops,adags)
print(hamMat)
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

np.isreal(ens).all()

ens

a^{\dagger}_{0}*a_{0} + a_{0}*a^{\dagger}_{0}
[[1.00000000000000 0 0 0 0 0 0 0]
 [0 3.00000000000000 0 0 0 0 0 0]
 [0 0 5.00000000000000 0 0 0 0 0]
 [0 0 0 7.00000000000000 0 0 0 0]
 [0 0 0 0 9.00000000000000 0 0 0]
 [0 0 0 0 0 11.0000000000000 0 0]
 [0 0 0 0 0 0 13.0000000000000 0]
 [0 0 0 0 0 0 0 7.00000000000000]]


array([ 1.+0.j,  3.+0.j,  5.+0.j,  7.+0.j,  7.+0.j,  9.+0.j, 11.+0.j,
       13.+0.j], dtype=complex64)

In [ ]:
np.kron(1,np.array([[1,0],[0,1]]))

## Full boson part

In [ ]:
bosonNI=sp.expand( p[n]**2/(2*aLat) + (aLat/2)*((q[n+1]-q[n-1])/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(q[n])**2 + aLat*V(q[n])*(q[n+1]-q[n-1])/(2*aLat) )

ham=0
totHam=bosonNI+bosonI
totHam

In [ ]:
N=3
aVal=1

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {q[-1]: q[N-1], q[N]: q[0],
                          x[N]: x[0], xd[N]: xd[0],
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {q[-1]: 0, q[N]: 0,
                          x[N]: 0, xd[N]: 0
                         }

In [ ]:

for i in range(0,N):
    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

def potential(n):
    return 0

for n in range(0,N):
    ham=ham.subs(V(q[n]),potential(n))
ham=ham.subs(aLat,aVal).simplify().expand()
ham

In [ ]:
m=1

HOdofSubs = {}
for i in range(0,N):
    HOdofSubs[q[i]] = 0.5*sp.sqrt(2/m)*(a[i] + adag[i])
    HOdofSubs[p[i]] = complex(0,1)*sp.sqrt(2*m)*(adag[i] - a[i])/2
    
hoHam=sp.expand(ham.subs(HOdofSubs))
sp.nsimplify(hoHam,tolerance=1e-10)

In [ ]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N)

In [ ]:
hamMat.shape

In [ ]:
np.isreal(hamMat).all()

In [ ]:
(hamMat==np.matrix(hamMat).H).all()

In [ ]:
np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])

In [ ]:
m=convert_term_to_matrix(1.0*a[0]*a[1], 4, 3).astype(float)

In [ ]:
for row in m:
    print(row)